In [ ]:
if pool_name + '_sandwich.pickle' not in os.listdir():
    sandwich_state = list()
    block_savers = list()
    for block in tqdm(df_swap.block_number.unique()):
        flag = True #TO DELLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL
        temp = df_swap[df_swap.block_number==block] #Select all swaps in the block
        n, N = 0, len(temp) #n = current position, N = number of swaps in the block
        if N > 3:
            block_savers.append( block )
        while n+3 <= N: #If there are at least 3 swaps, this block could contain a sandwich attack
            front_candidate = temp.iloc[n]
            m = 0
            while (n+m < N-1) and (temp.iloc[n+m+1].Event == front_candidate.Event):
                m += 1
            # If the first and second swaps have different signs, the first is not a front candidate
            if m == 0:
                sandwich_state.append( ['Not', block] )
                flag = False #TO DELLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL
                n += 1
            elif n+m == N-1:
                # If the last swap is the same as the previous ones, there is no sandwich attack
                for _ in range(m+1):
                    n += 1
                    sandwich_state.append( ['Not', block] )
                    flag = False #TO DELLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL
            else:
                # If there are Trade, Trade x ?, -Trade, then a sandwich attack is possible
                back_candidate = temp.iloc[n+m+1]
                # To understand if the sandwich is real, compare the senders
                back_wallet = back_candidate.wallet
                while (back_wallet != front_candidate.wallet) and (m>0):
                    # If the senders are different, the current front candidate is not part of the sandwich.
                    # Thus, update its info
                    sandwich_state.append( ['Not', block] )
                    flag = False #TO DELLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL

                    # Now, update the front candidate and the potential victims
                    n += 1
                    front_candidate = temp.iloc[n]
                    m -= 1
                if m != 0: #If the senders are the same, the sandwich is real
                    sandwich_state.append( ['Front', block] )
                    for _ in range(m):
                        sandwich_state.append( ['Victim', block] )
                    sandwich_state.append( ['Back', block] )
                    flag = False #TO DELLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL
                    n += m+2
                else: #If no sandwich attacks are found, update the front candidate for the next while cycle.
                    pass #n += 1

        # Fill the remaining swaps with Not a sandwich
        for _ in range(N-n):
            sandwich_state.append( ['Not', block] )
            flag = False #TO DELLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL
        
        if flag:
            raise ValueError('MISSING BLOCK:', block)

    # Converting the sandwich_state list into a DataFrame
    sandwich_state = np.array(sandwich_state)
    sandwich_state = pd.DataFrame(
        np.concatenate([
            sandwich_state[:,:1],
            df_swap.block_number.values.reshape(-1,1),
            df_swap.log_index.values.reshape(-1,1),
            df_swap.tx_hash.values.reshape(-1,1)], axis=1), index=df_swap.index)
    sandwich_state.columns = ['generalized_sandwich', 'block_number', 'log_index', 'tx_hash']
    sandwich_state['wallet'] = df_swap.wallet

    # Filtering true sandwich attacks
    g_swaps = list()
    for block_number, group in sandwich_state.groupby('block_number'):
        # Detect if the bloock contains a generalized sandwich attack
        if 'Front' in group.generalized_sandwich.values:
            att_wallet = group[ group.generalized_sandwich=='Front' ].wallet.values[0]
            # Look for self sandwiches
            temp_swap = df_swap[ df_swap.block_number==block_number ]
            temp_swap = temp_swap[group.generalized_sandwich!='Not']
            # Self Sandwich
            if len(temp_swap.wallet.unique()) == 1:
                for label in group.generalized_sandwich:
                    if label != 'Not':
                        g_swaps.append(label+'_Self')
                    else:
                        g_swaps.append(label)
            else:
                temp = df[ df.block_number==block_number ]
                # Generalized Sandwich
                temp_mint = temp[ temp.Event=='Mint' ]
                temp_burn = temp[ temp.Event=='Burn' ]
                if (len(temp_mint)>0) and (att_wallet in temp_mint.wallet.values) and (len(temp_burn)>0) and (att_wallet in temp_burn.wallet.values):
                    for label in group.generalized_sandwich:
                        if label != 'Not':
                            g_swaps.append(label+'_Gen')
                        else:
                            g_swaps.append(label)
                # True Sandwich
                else:
                    for label in group.generalized_sandwich:
                        g_swaps.append(label)
        # If not, append 'Not' to the list
        else:
            for label in group.generalized_sandwich:
                g_swaps.append(label)

    sandwich_state['Labels'] = g_swaps
    with open(pool_name + '_sandwich.pickle', 'wb') as f:
        pickle.dump(sandwich_state, f)
else:
    with open(pool_name + '_sandwich.pickle', 'rb') as f:
        sandwich_state = pickle.load(f)

print('True sandwiches:', len(sandwich_state[sandwich_state.Labels == 'Front']))
print('Generalized sandwiches:', len(sandwich_state[sandwich_state.Labels == 'Front_Gen']))
print('Self sandwiches:', len(sandwich_state[sandwich_state.Labels == 'Front_Self']))
print('Tot sandwiches:', len(sandwich_state[sandwich_state.Labels == 'Front']) +\
      len(sandwich_state[sandwich_state.Labels == 'Front_Gen']) +\
        len(sandwich_state[sandwich_state.Labels == 'Front_Self']))